In [1]:
from ortools.sat.python import cp_model

In [2]:
# def read_input(file):
#     with open(file, 'r') as f:
#         [n, k] = [int(x) for x in f.readline().split()]
#         capacity = [int(x) for x in f.readline().split()]
#         route = []
#         for i in range(2 * n + 1):
#             route.append([int(x) for x in f.readline().split()])
#     return n, k, capacity, route
# n, k, bus_capacity, route = read_input('data.txt')
# m = 10 * n

# n, k = 5, 2
# bus_capacity = [2, 2]
# route = [[9, 3, 8, 10, 5, 3, 1, 7, 7, 9, 2], [1, 9, 5, 3, 5, 3, 9, 7, 2, 3, 7], [7, 7, 6, 2, 6, 1, 6, 10, 10, 6, 9], [5, 3, 7, 2, 7, 7, 6, 6, 4, 9, 6], [9, 3, 10, 10, 8, 7, 6, 2, 1, 10, 2], [3, 4, 8, 8, 10, 8, 9, 4, 1, 8, 6], [2, 6, 5, 2, 2, 9, 8, 3, 6, 9, 8], [9, 8, 1, 1, 5, 5, 6, 4, 5, 6, 2], [10, 4, 9, 8, 9, 10, 8, 1, 7, 2, 6], [7, 4, 3, 3, 1, 10, 10, 4, 8, 8, 6], [3, 2, 8, 10, 4, 4, 4, 4, 6, 5, 2]]
# m = 10 * n

n, k = 5, 3
bus_capacity = [2, 2, 2]
route = [[9, 3, 8, 10, 5, 3, 1, 7, 7, 9, 2], [1, 9, 5, 3, 5, 3, 9, 7, 2, 3, 7], [7, 7, 6, 2, 6, 1, 6, 10, 10, 6, 9], [5, 3, 7, 2, 7, 7, 6, 6, 4, 9, 6], [9, 3, 10, 10, 8, 7, 6, 2, 1, 10, 2], [3, 4, 8, 8, 10, 8, 9, 4, 1, 8, 6], [2, 6, 5, 2, 2, 9, 8, 3, 6, 9, 8], [9, 8, 1, 1, 5, 5, 6, 4, 5, 6, 2], [10, 4, 9, 8, 9, 10, 8, 1, 7, 2, 6], [7, 4, 3, 3, 1, 10, 10, 4, 8, 8, 6], [3, 2, 8, 10, 4, 4, 4, 4, 6, 5, 2]]
m = 10 * n
print(n)
print(k)
print(bus_capacity)
print(route)

5
3
[2, 2, 2]
[[9, 3, 8, 10, 5, 3, 1, 7, 7, 9, 2], [1, 9, 5, 3, 5, 3, 9, 7, 2, 3, 7], [7, 7, 6, 2, 6, 1, 6, 10, 10, 6, 9], [5, 3, 7, 2, 7, 7, 6, 6, 4, 9, 6], [9, 3, 10, 10, 8, 7, 6, 2, 1, 10, 2], [3, 4, 8, 8, 10, 8, 9, 4, 1, 8, 6], [2, 6, 5, 2, 2, 9, 8, 3, 6, 9, 8], [9, 8, 1, 1, 5, 5, 6, 4, 5, 6, 2], [10, 4, 9, 8, 9, 10, 8, 1, 7, 2, 6], [7, 4, 3, 3, 1, 10, 10, 4, 8, 8, 6], [3, 2, 8, 10, 4, 4, 4, 4, 6, 5, 2]]


In [3]:
model = cp_model.CpModel()
INF = float('infinity')
x = [[[model.NewIntVar(0, 1, 'x[' + str(p) + ', ' + str(i) + ', ' + str(j) + ']') for j in range(2 * n + 1)] for i in range(2 * n + 1)] for p in range(k)]
y = [model.NewIntVar(0, max(bus_capacity), f'y[{i}]') for i in range(2*n+1)]
z = [model.NewIntVar(0, k, f'z[{i}]') for i in range(2*n+1)]
w = [[model.NewIntVar(0, 2 * n + len(bus_capacity), f'w[{i}][{j}]') for j in range(2*n+1)] for i in range(k)]

In [4]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    #print intermediate solution
    def __init__(self,variables, f):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables= variables
        self.__solution_count= 0
        self.f = f
    def on_solution_callback(self):
        self.__solution_count+= 1
        print('-'*20, self.__solution_count, self.Value(self.f))
        for xk in self.__variables:
            for xki in xk:
                for xkij in xki:
                    print('%i'% (self.Value(xkij)), end = ' ')
                print()
            print('\t' + '-'*10)
    def solution_count(self):
        return self.__solution_count

In [5]:
A = []
for i in range(2 * n + 1):
    for j in range(2 * n + 1):
        if i != j and not (i<=n and j==0) and not(i==0 and j>=n+1) and not(i==j+n):
            A.append([i, j])
print(A)

[[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [2, 1], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [3, 1], [3, 2], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [3, 10], [4, 1], [4, 2], [4, 3], [4, 5], [4, 6], [4, 7], [4, 8], [4, 9], [4, 10], [5, 1], [5, 2], [5, 3], [5, 4], [5, 6], [5, 7], [5, 8], [5, 9], [5, 10], [6, 0], [6, 2], [6, 3], [6, 4], [6, 5], [6, 7], [6, 8], [6, 9], [6, 10], [7, 0], [7, 1], [7, 3], [7, 4], [7, 5], [7, 6], [7, 8], [7, 9], [7, 10], [8, 0], [8, 1], [8, 2], [8, 4], [8, 5], [8, 6], [8, 7], [8, 9], [8, 10], [9, 0], [9, 1], [9, 2], [9, 3], [9, 5], [9, 6], [9, 7], [9, 8], [9, 10], [10, 0], [10, 1], [10, 2], [10, 3], [10, 4], [10, 6], [10, 7], [10, 8], [10, 9]]


In [6]:
Ai = lambda x: [i for i,j in A if j == x]
Ao = lambda x: [j for i,j in A if i == x]

In [7]:
for i in range(1, 2 * n + 1):
    a = 0
    for p in range(k):
        for j in Ao(i):
            a += x[p][i][j]
    model.Add(a == 1)
    
for i in range(1, 2 * n + 1):
    a = 0
    for p in range(k):
        for j in Ai(i):
            a += x[p][j][i]
    model.Add(a == 1)

In [8]:
for p in range(k):
    for i in range(2 * n + 1):
        a = 0
        b = 0
        for j in Ao(i):
            a += x[p][i][j]
        for j in Ai(i):
            b -= x[p][j][i]
        model.Add(a + b == 0)

In [9]:
for p in range(k):
    a = 0
    for j in range(1, n + 1):
        a += x[p][0][j]
    model.Add(a == 1)
    b = 0
    for j in range(1, n + 1):
        b += x[p][j + n][0]
    model.Add(b == 1)

In [10]:
for p in range(k):
    for i, j in A:
        temp = model.NewBoolVar('temp')
        model.Add(x[p][i][j] == 1).OnlyEnforceIf(temp)
        model.Add(y[j] <= bus_capacity[p]).OnlyEnforceIf(temp)
        model.Add(x[p][i][j] != 1).OnlyEnforceIf(temp.Not())
        # model.Add(y[j] <= bus_capacity[p]).OnlyEnforceIf(temp)

In [11]:
for p in range(k):
    for i,j in A:
        if 0 < j <= n:
            temp = model.NewBoolVar('temp')
            model.Add(x[p][i][j] == 1).OnlyEnforceIf(temp)
            model.Add(y[j] == y[i] + 1).OnlyEnforceIf(temp)
            model.Add(w[p][j] == w[p][i] + 1).OnlyEnforceIf(temp)
            model.Add(x[p][i][j] != 1).OnlyEnforceIf(temp.Not())
        if j > n:
            temp = model.NewBoolVar('temp')
            model.Add(x[p][i][j] == 1).OnlyEnforceIf(temp)
            model.Add(y[j] == y[i] - 1).OnlyEnforceIf(temp)
            model.Add(w[p][j] == w[p][i] + 1).OnlyEnforceIf(temp)
            model.Add(x[p][i][j] != 1).OnlyEnforceIf(temp.Not())
    for i, j in A:
        if i!=0 and j!=0:
            temp = model.NewBoolVar('temp')
            model.Add(x[p][i][j] == 1).OnlyEnforceIf(temp)
            model.Add(z[j] == z[i]).OnlyEnforceIf(temp)
            model.Add(z[j] == p).OnlyEnforceIf(temp)
            model.Add(z[i] == p).OnlyEnforceIf(temp)
            model.Add(x[p][i][j] != 1).OnlyEnforceIf(temp.Not())
            # model.Add(z[j] != p).OnlyEnforceIf(temp.Not())
            # model.Add(z[i] != p).OnlyEnforceIf(temp.Not())
            
for i in range(1, n + 1):
    model.Add(z[i] == z[i + n])
    
for p in range(k):
    for i in range(1, n + 1):
        model.Add(w[p][i] <= w[p][i + n])
        

In [12]:
import time
start = time.time()
f = model.NewIntVar(0, 10*(2*n+1), 'objective func')
model.Add(f == sum([x[p][i][j]*route[i][j] for p in range(k) for i in range(2*n+1) for j in range(2*n+1)]))
model.Minimize(f)
solver = cp_model.CpSolver()
solver.Solve(model)
print(solver.Solve(model) == cp_model.OPTIMAL)

True


In [13]:
def findNext(q,i):
    for j in Ao(i):
        if solver.Value(x[q][i][j]) > 0:
            return j
def routes(q):
    s = '0 - '
    i = findNext(q,0)
    while i != 0:
        s = s + str(i) + ' - '
        i = findNext(q,i)
    s = s + str(i)
    return s

In [14]:
print(f'Optimal objective value = {solver.Value(f)}')
for p in range(k):
    print('route[',p,'] = ',routes(p), bus_capacity[p])
    # print('route[',p,'] = ')
    for i,j in A:
        if solver.Value(x[p][i][j]) > 0:
            print('(',i,'-',j,')', solver.Value(y[j]), 'w = ', solver.Value(w[p][i]), solver.Value(w[p][j]), 'z =', solver.Value(z[i]), solver.Value(z[j]), 'distance =', route[i][j])
            # print('(',i,'-',j,')', y[j].solution_value())
print(time.time() - start)

Optimal objective value = 56
route[ 0 ] =  0 - 2 - 4 - 7 - 3 - 8 - 9 - 0 2
( 0 - 2 ) 1 w =  0 1 z = 0 0 distance = 8
( 2 - 4 ) 2 w =  1 2 z = 0 0 distance = 6
( 3 - 8 ) 1 w =  4 5 z = 0 0 distance = 4
( 4 - 7 ) 1 w =  2 3 z = 0 0 distance = 2
( 7 - 3 ) 2 w =  3 4 z = 0 0 distance = 1
( 8 - 9 ) 0 w =  5 6 z = 0 0 distance = 2
( 9 - 0 ) 0 w =  6 0 z = 0 0 distance = 7
route[ 1 ] =  0 - 5 - 10 - 0 2
( 0 - 5 ) 1 w =  0 1 z = 0 1 distance = 3
( 5 - 10 ) 0 w =  1 2 z = 1 1 distance = 6
( 10 - 0 ) 0 w =  2 0 z = 1 0 distance = 3
route[ 2 ] =  0 - 1 - 6 - 0 2
( 0 - 1 ) 1 w =  0 1 z = 0 2 distance = 3
( 1 - 6 ) 0 w =  1 2 z = 2 2 distance = 9
( 6 - 0 ) 0 w =  2 0 z = 2 0 distance = 2
0.8785109519958496
